# Thamos Advise Workflow

---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Create-the-Workflow" data-toc-modified-id="Create-the-Workflow-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Create the Workflow</a></span></li><li><span><a href="#Submit-the-Workflow" data-toc-modified-id="Submit-the-Workflow-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Submit the Workflow</a></span></li></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from argo.workflows import models

from argo.workflows.sdk import Workflow
from argo.workflows.sdk.tasks import *
from argo.workflows.sdk.templates import *

---

## Create the Workflow
w/ Argo Python SDK

In [3]:
REPOSITORY_ARTIFACT="/mnt/inputs/artifacts/repository"
"""A Path to repository artifact."""

WEBHOOK_RECEIVER_URL="http://macermak.aicoe.ultrahook.com/kebechet/"
"""Webhook receiver endpoint URL."""

'Webhook receiver endpoint URL.'

In [4]:
class ThothAdviseCheckRun(Workflow):
    
    arguments = V1alpha1Arguments(
        parameters=[
            V1alpha1Parameter(name="event", default="thoth_thamos_advise"),
            V1alpha1Parameter(name="check_run_id"),
            V1alpha1Parameter(name="installation"),
            V1alpha1Parameter(name="repo"),
            V1alpha1Parameter(name="revision", default="master"),
            V1alpha1Parameter(name="finished_webhook", default=WEBHOOK_RECEIVER_URL),
        ]
    )
    
    service_account_name = "argo"
    
    volumes = [
        models.V1Volume(name="cache", empty_dir={})
    ]
    
    @property
    def id(self) -> str:
        """Get Workflow ID."""
        prefix: str = self.name or getattr(self.metadata, "generate_name")
        digest: str = str(abs(self.__hash__()))
        return f"{prefix}-{digest}"
    
    @task
    def thamos_advise(self) -> V1alpha1Template:
        return self.thamos_advise_template()
    
    @closure(
        image="quay.io/cermakm/thamos:latest",
        resources=models.V1ResourceRequirements(
            limits={"memory": "128Mi", "cpu": "100m"},
        ),
        volume_mounts=[
            models.V1VolumeMount(name="cache", mount_path="/mnt/outputs/parameters")
        ]
    )
    @inputs.artifact(
        git=models.V1alpha1GitArtifact(
            repo="{{workflow.parameters.repo}}",
            revision="{{workflow.parameters.revision}}"
        ),
        name="repository",
        path=REPOSITORY_ARTIFACT
    )
    @outputs.parameter(
        name="payload",
        global_name="payload",
        value_from={"path": "/mnt/outputs/parameters/payload"}
    )
    def thamos_advise_template():
        import logging
        
        import json
        import os
        import subprocess
        
        from pathlib import Path
            
        logging.getLogger("thamos").setLevel("DEBUG")
        
        os.chdir("{{inputs.artifacts.repository.path}}")
        
        subprocess.call(["thamos", "config", "-I"])
        subprocess.call(["thamos", "advise", "--no-write"])
        
        analysis_id: str = Path(".thoth_last_analysis_id").read_text()
        
        payload: str = json.dumps({
            "analysis-id": analysis_id
        })
        
        Path("/mnt/outputs/parameters/payload").write_text(payload)
        
            
            
    @task
    @dependencies(["thamos-advise"])
    def trigger_finished_webhook(self) -> V1alpha1Template:
        return self.trigger_finished_webhook_template()
    
    @closure(
        scope="webhook",
        image="quay.io/cermakm/octomachinery:latest",
        resources=models.V1ResourceRequirements(
            limits={"memory": "128Mi", "cpu": "100m"},
        ),
        env=[
            models.V1EnvVar(
                name="WEBHOOK_SECRET",
                value_from=models.V1EnvVarSource(secret_key_ref={"name": "qeb-hwt-github-app", "key": "WEBHOOK_SECRET"})
            ),
        ]
    )
    def trigger_finished_webhook_template():
        import os
        import requests
        import uuid
        
        uuid = str(uuid.uuid4())
        
        data: dict = {
            "event": '{{workflow.parameters.event}}',
            "check_run_id": '{{workflow.parameters.check_run_id}}',   
            "installation": {
                "id": int('{{workflow.parameters.installation}}')
            },   
            "repo": '{{workflow.parameters.repo}}',
            "revision": '{{workflow.parameters.revision}}',
            "payload": json.loads('{{workflow.outputs.parameters.payload}}')
        }
            
        signature: str = webhook.get_signature(data, key=os.environ["WEBHOOK_SECRET"])
            
        headers={
            "Accept": "application/vnd.github.antiope-preview+json",
            "Content-Type": "application/json",
            "User-Agent": 'Workflow/{{workflow.name}}',
            "X-GitHub-Delivery": uuid,
            "X-GitHub-Event": '{{workflow.parameters.event}}',
            "X-Hub-Signature": f"sha1={signature}"
        }
        
        print("Headers:\n", headers)
        print("Payload:\n", data)
        
        response = requests.post('{{workflow.parameters.finished_webhook}}', json=data, headers=headers)
        response.raise_for_status()
        
    @scope("webhook")
    def get_signature(data: bytes, key: str) -> str:
        import hmac
        import json
        
        if not isinstance(data, bytes):
            data = json.dumps(data).encode("UTF-8")
            
        secret = key.encode("UTF-8")
        signature = hmac.new(secret, data, digestmod="sha1")
        
        return signature.hexdigest()
        
    
    
wf = ThothAdviseCheckRun()
print(wf.to_yaml())

api_version: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generate_name: thoth-advise-check-run-
  name: thoth-advise-check-run
spec:
  arguments:
    parameters:
    - default: thoth_thamos_advise
      name: event
    - name: check_run_id
    - name: installation
    - name: repo
    - default: master
      name: revision
    - default: http://macermak.aicoe.ultrahook.com/kebechet/
      name: finished_webhook
  entrypoint: main
  service_account_name: argo
  templates:
  - dag:
      tasks:
      - name: thamos-advise
        template: thamos-advise-template
      - dependencies:
        - thamos-advise
        name: trigger-finished-webhook
        template: trigger-finished-webhook-template
    name: main
  - inputs:
      artifacts:
      - git:
          repo: '{{workflow.parameters.repo}}'
          revision: '{{workflow.parameters.revision}}'
        name: repository
        path: /mnt/inputs/artifacts/repository
    name: thamos-advise-template
    outputs:
      parameter

---

## Submit the Workflow
w/ Thoth Workflow manager

In [5]:
namespace = "macermak-thoth-dev"

In [6]:
%env THOTH_AMUN_INFRA_NAMESPACE      $namespace 
%env THOTH_AMUN_INSPECTION_NAMESPACE $namespace
%env THOTH_WORKFLOW_CONTEXT          true

env: THOTH_AMUN_INFRA_NAMESPACE=macermak-thoth-dev
env: THOTH_AMUN_INSPECTION_NAMESPACE=macermak-thoth-dev
env: THOTH_WORKFLOW_CONTEXT=true


In [7]:
from thoth.common import WorkflowManager

mgr = WorkflowManager(ocp_config={
    "kubernetes_verify_tls": False
})
api = mgr.api

Failed to load in cluster configuration, fallback to a local development setup: Service host/port is not set.
TLS verification when communicating with k8s/okd master is disabled


In [8]:
wf.name = f"{wf.metadata.generate_name}{str(abs(hash(wf)))[:8]}"

workflow_parameters = {
    "check_run_id": "test",
    "installation": "6143192",
    "repo": "https://github.com/Cermakm/test",
}

workflow_id = wf.submit(api, namespace, parameters=workflow_parameters)
workflow_id

'thoth-advise-check-run-72158451'

In [9]:
!argo watch $workflow_id

Name:                thoth-advise-check-run-72158451
Namespace:           macermak-thoth-dev
ServiceAccount:      argo
Status:              Running
Created:             Thu Jan 16 19:38:25 +0100 (3 seconds ago)
Started:             Thu Jan 16 19:38:25 +0100 (3 seconds ago)
Duration:            3 seconds
Parameters:          
  check_run_id:      test
  installation:      6143192
  repo:              https://github.com/Cermakm/test
  event:             thoth_thamos_advise
  revision:          master
  finished_webhook:  http://macermak.aicoe.ultrahook.com/kebechet/

STEP                                         PODNAME                                     DURATION  MESSAGE
 ● thoth-advise-check-run-72158451 (main)                                                          
 └-◷ thamos-advise (thamos-advise-template)  thoth-advise-check-run-72158451-4095013483  3s        PodInitializing
Name:                thoth-advise-check-run-72158451
Namespace:           macermak-thoth-dev
ServiceAccoun

Name:                thoth-advise-check-run-72158451
Namespace:           macermak-thoth-dev
ServiceAccount:      argo
Status:              Running
Created:             Thu Jan 16 19:38:25 +0100 (11 seconds ago)
Started:             Thu Jan 16 19:38:25 +0100 (11 seconds ago)
Duration:            11 seconds
Parameters:          
  check_run_id:      test
  installation:      6143192
  repo:              https://github.com/Cermakm/test
  event:             thoth_thamos_advise
  revision:          master
  finished_webhook:  http://macermak.aicoe.ultrahook.com/kebechet/

STEP                                         PODNAME                                     DURATION  MESSAGE
 ● thoth-advise-check-run-72158451 (main)                                                          
 └-● thamos-advise (thamos-advise-template)  thoth-advise-check-run-72158451-4095013483  11s       
Name:                thoth-advise-check-run-72158451
Namespace:           macermak-thoth-dev
ServiceAccount:      argo

Name:                thoth-advise-check-run-72158451
Namespace:           macermak-thoth-dev
ServiceAccount:      argo
Status:              Running
Created:             Thu Jan 16 19:38:25 +0100 (20 seconds ago)
Started:             Thu Jan 16 19:38:25 +0100 (20 seconds ago)
Duration:            20 seconds
Parameters:          
  check_run_id:      test
  installation:      6143192
  repo:              https://github.com/Cermakm/test
  event:             thoth_thamos_advise
  revision:          master
  finished_webhook:  http://macermak.aicoe.ultrahook.com/kebechet/

STEP                                         PODNAME                                     DURATION  MESSAGE
 ● thoth-advise-check-run-72158451 (main)                                                          
 └-● thamos-advise (thamos-advise-template)  thoth-advise-check-run-72158451-4095013483  20s       
Name:                thoth-advise-check-run-72158451
Namespace:           macermak-thoth-dev
ServiceAccount:      argo

Name:                thoth-advise-check-run-72158451
Namespace:           macermak-thoth-dev
ServiceAccount:      argo
Status:              Running
Created:             Thu Jan 16 19:38:25 +0100 (29 seconds ago)
Started:             Thu Jan 16 19:38:25 +0100 (29 seconds ago)
Duration:            29 seconds
Parameters:          
  check_run_id:      test
  installation:      6143192
  repo:              https://github.com/Cermakm/test
  event:             thoth_thamos_advise
  revision:          master
  finished_webhook:  http://macermak.aicoe.ultrahook.com/kebechet/

STEP                                         PODNAME                                     DURATION  MESSAGE
 ● thoth-advise-check-run-72158451 (main)                                                          
 └-● thamos-advise (thamos-advise-template)  thoth-advise-check-run-72158451-4095013483  29s       
Name:                thoth-advise-check-run-72158451
Namespace:           macermak-thoth-dev
ServiceAccount:      argo

Name:                thoth-advise-check-run-72158451
Namespace:           macermak-thoth-dev
ServiceAccount:      argo
Status:              Running
Created:             Thu Jan 16 19:38:25 +0100 (38 seconds ago)
Started:             Thu Jan 16 19:38:25 +0100 (38 seconds ago)
Duration:            38 seconds
Parameters:          
  check_run_id:      test
  installation:      6143192
  repo:              https://github.com/Cermakm/test
  event:             thoth_thamos_advise
  revision:          master
  finished_webhook:  http://macermak.aicoe.ultrahook.com/kebechet/

STEP                                         PODNAME                                     DURATION  MESSAGE
 ● thoth-advise-check-run-72158451 (main)                                                          
 └-● thamos-advise (thamos-advise-template)  thoth-advise-check-run-72158451-4095013483  38s       
Name:                thoth-advise-check-run-72158451
Namespace:           macermak-thoth-dev
ServiceAccount:      argo

...

Name:                thoth-advise-check-run-72158451
Namespace:           macermak-thoth-dev
ServiceAccount:      argo
Status:              Running
Created:             Thu Jan 16 19:38:25 +0100 (50 seconds ago)
Started:             Thu Jan 16 19:38:25 +0100 (50 seconds ago)
Duration:            50 seconds
Parameters:          
  check_run_id:      test
  installation:      6143192
  repo:              https://github.com/Cermakm/test
  event:             thoth_thamos_advise
  revision:          master
  finished_webhook:  http://macermak.aicoe.ultrahook.com/kebechet/
Output Parameters:   
  payload:           {"analysis-id": "adviser-7fe9e305"}

STEP                                                               PODNAME                                     DURATION  MESSAGE
 ● thoth-advise-check-run-72158451 (main)                                                                                
 ├-✔ thamos-advise (thamos-advise-template)                        thoth-advise-check-run-72158

Name:                thoth-advise-check-run-72158451
Namespace:           macermak-thoth-dev
ServiceAccount:      argo
Status:              Running
Created:             Thu Jan 16 19:38:25 +0100 (57 seconds ago)
Started:             Thu Jan 16 19:38:25 +0100 (57 seconds ago)
Duration:            57 seconds
Parameters:          
  check_run_id:      test
  installation:      6143192
  repo:              https://github.com/Cermakm/test
  event:             thoth_thamos_advise
  revision:          master
  finished_webhook:  http://macermak.aicoe.ultrahook.com/kebechet/
Output Parameters:   
  payload:           {"analysis-id": "adviser-7fe9e305"}

STEP                                                               PODNAME                                     DURATION  MESSAGE
 ● thoth-advise-check-run-72158451 (main)                                                                                
 ├-✔ thamos-advise (thamos-advise-template)                        thoth-advise-check-run-72158

Name:                thoth-advise-check-run-72158451
Namespace:           macermak-thoth-dev
ServiceAccount:      argo
Status:              Running
Created:             Thu Jan 16 19:38:25 +0100 (1 minute ago)
Started:             Thu Jan 16 19:38:25 +0100 (1 minute ago)
Duration:            1 minute 4 seconds
Parameters:          
  check_run_id:      test
  installation:      6143192
  repo:              https://github.com/Cermakm/test
  event:             thoth_thamos_advise
  revision:          master
  finished_webhook:  http://macermak.aicoe.ultrahook.com/kebechet/
Output Parameters:   
  payload:           {"analysis-id": "adviser-7fe9e305"}

STEP                                                               PODNAME                                     DURATION  MESSAGE
 ● thoth-advise-check-run-72158451 (main)                                                                                
 ├-✔ thamos-advise (thamos-advise-template)                        thoth-advise-check-run-7

Name:                thoth-advise-check-run-72158451
Namespace:           macermak-thoth-dev
ServiceAccount:      argo
Status:              Running
Created:             Thu Jan 16 19:38:25 +0100 (1 minute ago)
Started:             Thu Jan 16 19:38:25 +0100 (1 minute ago)
Duration:            1 minute 10 seconds
Parameters:          
  check_run_id:      test
  installation:      6143192
  repo:              https://github.com/Cermakm/test
  event:             thoth_thamos_advise
  revision:          master
  finished_webhook:  http://macermak.aicoe.ultrahook.com/kebechet/
Output Parameters:   
  payload:           {"analysis-id": "adviser-7fe9e305"}

STEP                                                               PODNAME                                     DURATION  MESSAGE
 ● thoth-advise-check-run-72158451 (main)                                                                                
 ├-✔ thamos-advise (thamos-advise-template)                        thoth-advise-check-run-

Name:                thoth-advise-check-run-72158451
Namespace:           macermak-thoth-dev
ServiceAccount:      argo
Status:              Running
Created:             Thu Jan 16 19:38:25 +0100 (1 minute ago)
Started:             Thu Jan 16 19:38:25 +0100 (1 minute ago)
Duration:            1 minute 17 seconds
Parameters:          
  check_run_id:      test
  installation:      6143192
  repo:              https://github.com/Cermakm/test
  event:             thoth_thamos_advise
  revision:          master
  finished_webhook:  http://macermak.aicoe.ultrahook.com/kebechet/
Output Parameters:   
  payload:           {"analysis-id": "adviser-7fe9e305"}

STEP                                                               PODNAME                                     DURATION  MESSAGE
 ● thoth-advise-check-run-72158451 (main)                                                                                
 ├-✔ thamos-advise (thamos-advise-template)                        thoth-advise-check-run-

---